In [7]:
import simpy
import numpy as np
import pandas as pd

In [8]:
#Parameter definieren
Klinikum_sud =  961                                 #Alle Betten in Klinikum Süd Nürnberg
Klinikum_nord = 1272                                #Alle Betten in Klinikum Nord Nürnberg
Actual_Beds = Klinikum_sud + Klinikum_nord

ActualICU_BEDS = 158                               #die ingesamte Anzahl an Intensivbetten im Klinikum Nürnberg
ActualNORM_BEDS = Actual_Beds - ActualICU_BEDS     #die insgesamte Anzahl an Normalstationsbetten im Klinikum Nürnberg


ICU_BEDS = int(ActualICU_BEDS * 30 / 100)          #30 Prozent der verfügbare Intensivbetten Für Corona Patienten in Nürnberg
NORM_BEDS = int(ActualNORM_BEDS * 25 / 100)          #25 Prozent der verfügbare Betten für Corona Patienten auf Normalstationen in Nürnberg
TOTAL_BEDS = ICU_BEDS + NORM_BEDS               #Komplete Bettkapazität für Corona Patienten


MEAN_LOS = 14.0          # Mittelwert der Liegedauer in Tagen
BASE_SEED = 42
R = 20                   # Replikationen
SIM_DURATION = 150       # Simulationszeitraum in Tagen


In [9]:
#Simulation eines Patienten vorerst wird eine Vorbelastung geschaffen und dann fangen die echten Patienten an

def patient_process(env, hospital, mean_los, rng, wait_times, is_preloaded=False):
    """
    Simuliert den diskreten Ereignisablauf eines Patienten (Entity) im System.

    Input-Kriterien & Parameter:
    ---------------------------
    env : simpy.Environment
        Die zentrale Instanz der Simulation, die die virtuelle Zeit (env.now) verwaltet.
    hospital : simpy.Resource
        Die begrenzte Ressource 'Bett'. Repräsentiert die Systemkapazität (TOTAL_BEDS).
        Ein Patient muss diese anfordern (request), bevor die Behandlung beginnt.
    mean_los : float
        Der Input für die mittlere Verweildauer (Mean Length of Stay). 
        Dient als Skalenparameter für die Exponentialverteilung.
    rng : numpy.random.Generator
        Der lokalisierte Zufallsgenerator. Verarbeitet den Seed der jeweiligen 
        Replikation, um die Varianz in der Aufenthaltsdauer zu erzeugen.
    wait_times : list
        Der Zielspeicher (Output-Container). Hier wird die Differenz zwischen 
        Ankunft und Aufnahme (t_aufnahme - t_ankunft) für die spätere 
        statistische Auswertung abgelegt.
    is_preloaded : bool, optional (Default: False)
        Steuerungs-Input zur Vermeidung von Bias. Patienten, die bereits bei t=0
        im System sind (Vorbelastung), haben per Definition nicht gewartet.
        Falls True, erfolgt kein Eintrag in 'wait_times'.

    Prozess-Logik:
    --------------
    1. Registrierung der Ankunftszeit (Timestamping).
    2. Warteschlange: Der Prozess pausiert (yield), bis ein Bett frei wird.
    3. Belegung: Berechnung der Wartezeit und Start der Behandlung.
    4. Service-Zeit: Generierung einer Zufallszahl (Exponentialverteilung):
       f(x; 1/mu) = (1/mu) * e^(-(1/mu)x).
    5. Freigabe: Das Bett wird nach Ablauf der Zeit automatisch wieder freigegeben.
    """
    t_ankunft = env.now
    with hospital.request() as req:
        yield req
        t_aufnahme = env.now
        
        # Statistik nur für reguläre Patienten (Ausschluss der Vorbelastung)
        if not is_preloaded:
            wait_times.append(t_aufnahme - t_ankunft)
        
        yield env.timeout(rng.exponential(mean_los))


In [10]:
def setup_hospital(env, hospital, rng, wait_times, patients_per_week):
    """
    Steuert die Lastgenerierung basierend auf den Ankunftsraten.

    Input-Kriterien:
    - env: SimPy Environment (Zentraler Zeitgeber).
    - hospital: Resource-Objekt mit definierter Kapazität 'TOTAL_BEDS'.
    - rng: Ein numpy Random Generator. Nutzt den BASE_SEED, um 
           stochastische Varianz zwischen Replikationen zu ermöglichen.
    - wait_times: Referenz auf eine Liste (Mutable), die als Datenspeicher 
                  für die Zielgröße 'Wartezeit' fungiert.

    Logik der Input-Verarbeitung:
    Die Funktion transformiert die Rate (Patients/Week) in einen 
    Interarrival-Zeitraum (1/lambda) für einen Poisson-Prozess.
    """
    mean_interarrival = 7.0 / patients_per_week
    
    # Vorbelastung (Steady-State): 75% der Betten zum Start belegen
    preload_count = int(TOTAL_BEDS * 0.75)
    for _ in range(preload_count):
        env.process(patient_process(env, hospital, MEAN_LOS, rng, wait_times, is_preloaded=True))
    
    # Kontinuierliche Ankünfte
    while True:
        yield env.timeout(rng.exponential(mean_interarrival))
        env.process(patient_process(env, hospital, MEAN_LOS, rng, wait_times))


In [11]:
def run_simulation(patients_per_week):
    """
    Führt eine einzelne, unabhängige Replikation des Simulationsmodells aus.

    Dieser Docstring beschreibt die Kapselung eines Szenarios, um eine 
    statistisch belastbare Stichprobe für die Monte-Carlo-Analyse zu generieren.

    Input-Kriterien & Szenario-Parameter:
    ------------------------------------
    num_beds : int
        Die Kapazitätsvariable (Entscheidungsvariable). Definiert das Szenario 
        hinsichtlich der Ressourcenverfügbarkeit. Änderungen an diesem Wert 
        erlauben "What-if"-Analysen (z. B. Auswirkung von Bettenabbau).
        
    seed : int
        Der deterministische Input für den Zufallsgenerator. 
        Wichtig: Jede Replikation eines Szenarios muss einen eindeutigen Seed 
        erhalten, um Korrelationen zwischen den Läufen zu vermeiden und 
        den Standardfehler (SE) korrekt berechnen zu können.

    Szenario-Logik & Ablauf:
    -----------------------
    1. Instanziierung: Erstellung einer isolierten simpy.Environment.
    2. Konfiguration: Festlegung der Kapazität basierend auf num_beds.
    3. Stochastik: Initialisierung des RNG (Random Number Generator) mit dem Seed,
       um die Reproduzierbarkeit des Szenarios zu gewährleisten.
    4. Simulation: Ausführung bis zum Erreichen von SIM_DURATION.
    5. Datengewinnung: Extraktion der Performance-Indikatoren (KPIs) für 
       genau diesen Pfad.

    Output-Kriterien (Szenario-Ergebnis):
    ------------------------------------
    return : tuple (float, float)
        Gibt die aggregierten Metriken der Replikation zurück:
        - Arithmetisches Mittel der Wartezeiten (Punkt-Schätzer).
        - 95%-Quantil der Wartezeiten (Risiko-Maß/Service-Level).
    """
    rep_means = []
    rep_q95s = []
    
    for i in range(R):
        # Saubere RNG-Nutzung 
        rng = np.random.default_rng(BASE_SEED + i)
        env = simpy.Environment()
        hospital = simpy.Resource(env, capacity=TOTAL_BEDS)
        wait_times = []
        
        env.process(setup_hospital(env, hospital, rng, wait_times, patients_per_week))
        env.run(until=SIM_DURATION)
        
        if wait_times:
            rep_means.append(np.mean(wait_times))
            rep_q95s.append(np.quantile(wait_times, 0.95))
            
    # Aggregation über die Replikate (Unsicherheitsquantifizierung)
    m_final = np.mean(rep_means)
    se_final = np.std(rep_means, ddof=1) / np.sqrt(R) # SE über Replikate
    q95_final = np.mean(rep_q95s)                    # Ø 95%-Quantil
    ci = (m_final - 1.96 * se_final, m_final + 1.96 * se_final)
    
    return m_final, se_final, ci, q95_final

# --- 2. Durchführung Szenarien-Vergleich ---
# 200/Woche = Normal, 285/Woche = Grenzbereich 300 = Überlastung 
szenarien = [200, 285, 300]
results = []

for p_week in szenarien:
    m, se, ci, q95 = run_simulation(p_week)
    rho = ((p_week / 7.0) * MEAN_LOS) / TOTAL_BEDS # Theoretische Auslastung
    
    results.append({
        "Szenario": "Normal" if p_week == 200 else "Stress-Test",
        "Pat./Woche": p_week,
        "Auslastung (ρ)": f"{rho*100:.1f}%",
        "Mean Wartezeit": f"{m:.4f} d",
        "Std.-Fehler (SE)": f"{se:.4f}",
        "95%-Konf.-Intervall": f"[{ci[0]:.3f}, {ci[1]:.3f}]",
        "Ø 95%-Quantil": f"{q95:.4f} d"
    })



In [12]:
#Output

print("\nERGEBNISSE DER KRANKENHAUS-SIMULATION (Weg A: Lastvariation)")
print("=" * 95)
df_output = pd.DataFrame(results)
print(df_output.to_string(index=False))
print("-" * 95)
print(f"Bedingungen: Betten = {TOTAL_BEDS}, Replikationen R = {R}, Dauer = {SIM_DURATION} Tage.")


ERGEBNISSE DER KRANKENHAUS-SIMULATION (Weg A: Lastvariation)
   Szenario  Pat./Woche Auslastung (ρ) Mean Wartezeit Std.-Fehler (SE) 95%-Konf.-Intervall Ø 95%-Quantil
     Normal         200          70.8%       0.0000 d           0.0000      [0.000, 0.000]      0.0000 d
Stress-Test         285         100.9%       1.0425 d           0.1961      [0.658, 1.427]      2.5385 d
Stress-Test         300         106.2%       3.1533 d           0.2910      [2.583, 3.724]      6.8037 d
-----------------------------------------------------------------------------------------------
Bedingungen: Betten = 565, Replikationen R = 20, Dauer = 150 Tage.
